### 과적합 문제와 그 해결방법 ###
데이터셋: sonar.csv
[고먼(Gorman)과 세즈노프스키(Sejnowski) (1988)]

광석과 일반 돌에 대해 음파 탐지기를 쏜 후 그 결과를 정리한 데이터를 바탕으로 광석과 일반 돌을 구분하는 문제
* 사례 수: 208
* 속성[설명변수] 수: 60
* 관심변수(범주형변수, Binary Classification): 2 (R, M) 

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

# 데이터 입력
df = pd.read_csv('sonar.csv', header=None)

In [3]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [6]:
df.iloc[:, 60].unique()

array(['R', 'M'], dtype=object)

In [7]:
dataset = df.values
X = dataset[:, 0:60].astype(float)
Y_obj = dataset[:, 60]
Y_obj[0:5,]

array(['R', 'R', 'R', 'R', 'R'], dtype=object)

In [8]:
# Categorical 데이터를 Numerical 데이터로 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)
Y[0:5,]

array([1, 1, 1, 1, 1])

In [9]:
# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim = 60, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                1464      
                                                                 
 dense_1 (Dense)             (None, 10)                250       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,725
Trainable params: 1,725
Non-trainable params: 0
_________________________________________________________________


In [11]:
# 모델 컴파일
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
# 모델 실행
model.fit(X, Y, epochs = 200, batch_size = 5) 

Epoch 1/200
42/42 [==============================] - 2s 4ms/step - loss: 0.2468 - accuracy: 0.5096
Epoch 2/200
42/42 [==============================] - 0s 5ms/step - loss: 0.2269 - accuracy: 0.6971
Epoch 3/200
42/42 [==============================] - 0s 5ms/step - loss: 0.2153 - accuracy: 0.7115
Epoch 4/200
42/42 [==============================] - 0s 5ms/step - loss: 0.2048 - accuracy: 0.7115
Epoch 5/200
42/42 [==============================] - 0s 5ms/step - loss: 0.1969 - accuracy: 0.7260
Epoch 6/200
42/42 [==============================] - 0s 4ms/step - loss: 0.1871 - accuracy: 0.7692
Epoch 7/200
42/42 [==============================] - 0s 4ms/step - loss: 0.1768 - accuracy: 0.7788
Epoch 8/200
42/42 [==============================] - 0s 5ms/step - loss: 0.1715 - accuracy: 0.7837
Epoch 9/200
42/42 [==============================] - 0s 4ms/step - loss: 0.1620 - accuracy: 0.8125
Epoch 10/200
42/42 [==============================] - 0s 5ms/step - loss: 0.1619 - accuracy: 0.7981
Epoch 11/

In [13]:
# 결과 출력
score = model.evaluate(X, Y)
print(model.metrics_names)

7/7 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 1.0000
['loss', 'accuracy']


In [14]:
print("\n Loss: %.4f" % (score[0]))


 Loss: 0.0034


In [15]:
print("\n Accuracy: %.4f" % (score[1]))


 Accuracy: 1.0000


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

In [17]:
X_train.shape, X_test.shape

((145, 60), (63, 60))

In [18]:
len(X_train) / len(X), len(X_test) / len(X)

(0.6971153846153846, 0.30288461538461536)

In [19]:
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                1464      
                                                                 
 dense_4 (Dense)             (None, 10)                250       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,725
Trainable params: 1,725
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(X_train, Y_train, epochs=130, batch_size=5)

Epoch 1/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2506 - accuracy: 0.4690
Epoch 2/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2418 - accuracy: 0.5793
Epoch 3/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2364 - accuracy: 0.6207
Epoch 4/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2301 - accuracy: 0.6621
Epoch 5/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2242 - accuracy: 0.6828
Epoch 6/130
29/29 [==============================] - 0s 3ms/step - loss: 0.2172 - accuracy: 0.6966
Epoch 7/130
29/29 [==============================] - 0s 2ms/step - loss: 0.2104 - accuracy: 0.7310
Epoch 8/130
29/29 [==============================] - 0s 3ms/step - loss: 0.2086 - accuracy: 0.6897
Epoch 9/130
29/29 [==============================] - 0s 3ms/step - loss: 0.1972 - accuracy: 0.7103
Epoch 10/130
29/29 [==============================] - 0s 2ms/step - loss: 0.1917 - accuracy: 0.7517
Epoch 11/

In [23]:
# 테스트셋에 모델 적용
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

2/2 [==============================] - 0s 7ms/step - loss: 0.1369 - accuracy: 0.8254

 Test Accuracy: 0.8254


In [24]:
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [25]:
# 빈 accuracy 배열
accuracy = []

# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

# 결과 출력
print("\n %.f fold accuracy:" % n_fold, accuracy)

Epoch 1/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2469 - accuracy: 0.5348
Epoch 2/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2407 - accuracy: 0.5348
Epoch 3/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2361 - accuracy: 0.5508
Epoch 4/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2314 - accuracy: 0.6043
Epoch 5/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2257 - accuracy: 0.5668
Epoch 6/100
38/38 [==============================] - 0s 3ms/step - loss: 0.2207 - accuracy: 0.6578
Epoch 7/100
38/38 [==============================] - 0s 2ms/step - loss: 0.2147 - accuracy: 0.6845
Epoch 8/100
38/38 [==============================] - 0s 2ms/step - loss: 0.2071 - accuracy: 0.7166
Epoch 9/100
38/38 [==============================] - 0s 3ms/step - loss: 0.1993 - accuracy: 0.7326
Epoch 10/100
38/38 [==============================] - 0s 3ms/step - loss: 0.1890 - accuracy: 0.7807
Epoch 11/

In [26]:
np.array(accuracy, dtype=np.float32).mean()

0.83237